In [ ]:
!pip install pydicom

In [ ]:
import os
import numpy as np
import cv2
import pydicom
from pydicom.tag import Tag
from matplotlib import pyplot as plt

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
dcm_path = "drive/MyDrive/dataset/ct_dcmdir/Brain01"
data = pydicom.dcmread(dcm_path)

print(data)

In [ ]:
pn = data.PatientName
wc = data.WindowCenter
ww = data.WindowWidth
img = data.pixel_array
print(f"Patient Name: {pn}")
print(f"Window Center: {wc}")
print(f"Window Width: {ww}")
print(f"Pixel Array: {img, img.shape}")

In [ ]:
max = wc + ww / 2
min = wc - ww / 2
print(f"max = {max} min = {min}")

plt.imshow(img, cmap="gray", vmax=max, vmin=min)
plt.axis("off")
plt.show()

In [ ]:
_wc = 0
_ww = 200
max = _wc + _ww / 2
min = _wc - _ww / 2
print(f"max = {max} min = {min}")

plt.imshow(img, cmap="gray", vmax=max, vmin=min)
plt.axis("off")
plt.show()

In [ ]:
print("SOP Instance UID  :",Tag("SOPInstanceUID"), data.SOPInstanceUID)
print("Study Date        :",Tag("StudyDate"),      data.StudyDate)
print("Study Time        :",Tag("StudyTime"),      data.StudyTime)
print("Modality          :",Tag("Modality"),       data.Modality)

In [ ]:
print("Patient Name :", Tag("PatientName"), data.PatientName)
print("Patient ID   :", Tag("PatientID"),   data.PatientID)

In [ ]:
print("Body Part Examined :", Tag("BodyPartExamined"), data.BodyPartExamined)
print("Slice Thickness    :", Tag("SliceThickness"), data.SliceThickness)
print("Convolution Kernel :", Tag("ConvolutionKernel"), data.ConvolutionKernel)

In [ ]:
print("Study Instance UID      :", Tag("StudyInstanceUID"), data.StudyInstanceUID)
print("Series Instance UID     :", Tag("SeriesInstanceUID"), data.SeriesInstanceUID)
print("Series Number           :", Tag("SeriesNumber"), data.SeriesNumber)
print("Image Position (Patient):", Tag("ImagePositionPatient"), data.ImagePositionPatient)
print("Acquisition Number      :", Tag("AcquisitionNumber"), data.AcquisitionNumber)
print("Instance Number         :", Tag("InstanceNumber"), data.InstanceNumber)
print("Slice Location          :", Tag("SliceLocation"), data.SliceLocation)

In [ ]:
print("Photometric Interpretation :", Tag("PhotometricInterpretation"), data.PhotometricInterpretation)
print("Rows                       :", Tag("Rows"), data.Rows)
print("Columns　　                :", Tag("Columns"), data.Columns)
print("Pixel Spacing　　　　　　  :", Tag("PixelSpacing"), data.PixelSpacing)
print("Bits Allocated             :", Tag("BitsAllocated"), data.BitsAllocated)
print("Bits Stored                :", Tag("BitsStored"), data.BitsStored)
print("High Bit                   :", Tag("HighBit"), data.HighBit)
print("Pixel Representation       :", Tag("PixelRepresentation"), data.PixelRepresentation)
print("Window Center              :", Tag("WindowCenter"), data.WindowCenter)
print("Window Width               :", Tag("WindowWidth"), data.WindowWidth)
print("Rescale Intercept          :", Tag("RescaleIntercept"), data.RescaleIntercept)
print("Rescale Slope              :", Tag("RescaleSlope"),   data.RescaleSlope)

In [ ]:
print("Pixel Data:", data.pixel_array)
print(data.pixel_array.shape)

In [ ]:
# 匿名化
orgdir = "drive/My Drive/dataset/ct_dcmdir/"
dstdir = "drive/My Drive/result/"

if not os.path.isdir(dstdir):
    os.mkdir(dstdir)

dummy_info = [['00100010', 'no name'],
              ['00100020', 'no ptid']]

files = os.listdir(orgdir)  # orgdir内のすべてのファイル名を取得
print(files)

for i, name in enumerate(files):
    path = orgdir + name
    data = pydicom.dcmread(path)            # DICOM画像の読み込み
    for j, info in enumerate(dummy_info):
        tag = info[0]
        value = info[1]
        data[tag].value = value             # 情報の置き換え
    save_name = f"{dstdir}anony_{name}"     # 保存ファイル名
    pydicom.dcmwrite(save_name, data)       # DICOM画像の書き込み
    print(data["00100010"])                 # 患者名を表示する

In [ ]:
# リスケール処理と階調処理
dcm_path = "drive/MyDrive/dataset/ct_dcmdir/Brain01"
data = pydicom.dcmread(dcm_path)

wc = data.WindowCenter
ww = data.WindowWidth
ri = data.RescaleIntercept
rs = data.RescaleSlope
img = data.pixel_array

print(wc, ww, ri, rs, img)

ct = img * rs + ri
max = wc + ww / 2
min = wc - ww / 2

# 8 bit
res = 255 * (ct - min) / (max - min)
res = np.clip(res, 0, 255)
cv2.imwrite("drive/MyDrive/res_8bit.png", res)

plt.imshow(res, cmap="gray")
plt.axis("off")
plt.title("8bit")
plt.show()

# 16 bit
res = 65535 * (ct - min) / (max - min)
res = np.clip(res, 0, 65535)
cv2.imwrite("drive/MyDrive/res_16bit.png", res)

plt.imshow(res, cmap="gray")
plt.axis("off")
plt.title("16bit")
plt.show()

res = res.astype(np.uint16)
cv2.imwrite("drive/MyDrive/res_16bit_casted.png", res)